In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()
x_train = x_train/255.
y_train = y_train/255.
y_train_one_hot = tf.squeeze(tf.one_hot(y_train,10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test,10),axis=1)


def conv_block(x,filters,s):
    f1, f2, f3 = filters
    x_shortcut = x
    
    x = tf.keras.layers.Conv2D(filters=f1,kernel_size=1,strides=s,padding='valid',
                              kernel_initializer=tf.keras.initializers.glorot_uniform())(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters=f2,kernel_size=3,strides=1,padding='same',
                              kernel_initializer=tf.keras.initializers.glorot_uniform())(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters=f3,kernel_size=1,strides=1,padding='valid',
                              kernel_initializer=tf.keras.initializers.glorot_uniform())(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x_shortcut = tf.keras.layers.Conv2D(filters=f3,kernel_size=1,strides=s,padding='valid',
                                       kernel_initializer=tf.keras.initializers.glorot_uniform())(x_shortcut)
    x_shortcut = tf.keras.layers.BatchNormalization(axis=3)(x_shortcut)
    
    x = tf.keras.layers.Add()([x,x_shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    
    return x

    
def id_block(x,filters):
    f1, f2, f3 = filters
    x_shortcut = x
    
    x = tf.keras.layers.Conv2D(filters=f1,kernel_size=1,strides=1,padding='valid',
                              kernel_initializer=tf.keras.initializers.glorot_uniform())(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters=f2,kernel_size=3,strides=1,padding='same',
                              kernel_initializer=tf.keras.initializers.glorot_uniform())(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters=f3,kernel_size=1,strides=1,padding='valid',
                              kernel_initializer=tf.keras.initializers.glorot_uniform())(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Add()([x,x_shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    
    return x

def ResNet50():
    x_input = tf.keras.layers.Input((32,32,3))
    
    x = tf.keras.layers.ZeroPadding2D((3,3))(x_input)
    
    x = tf.keras.layers.Conv2D(filters=64,kernel_size=7,strides=2,
                              kernel_initializer=tf.keras.initializers.glorot_uniform())(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((3,3),strides=(2,2))(x)    

    x = conv_block(x,[64,64,256],s=1)
    x = id_block(x,[64,64,256])
    x = id_block(x,[64,64,256])
    
    x = conv_block(x,[128,128,256],s=2)
    x = id_block(x,[128,128,256])
    x = id_block(x,[128,128,256])
    x = id_block(x,[128,128,256])    
    
    x = conv_block(x,[256,256,1024],s=2)
    x = id_block(x,[256,256,1024])
    x = id_block(x,[256,256,1024])    
    x = id_block(x,[256,256,1024])
    x = id_block(x,[256,256,1024])
    x = id_block(x,[256,256,1024]) 
    
    x = conv_block(x,[512,512,2048],s=2)
    x = id_block(x,[512,512,2048])
    x = id_block(x,[512,512,2048])
    
    x = tf.keras.layers.AveragePooling2D(padding='same')(x)
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(10,activation='softmax',
                             kernel_initializer=tf.keras.initializers.glorot_uniform())(x)
    
    model = tf.keras.models.Model(inputs=x_input,outputs=x,name='resnet-50')
    
    return model

model = ResNet50()
model.summary()

Model: "resnet-50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 38, 38, 3)    0           input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_109 (Conv2D)             (None, 16, 16, 64)   9472        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
batch_normalization_106 (BatchN (None, 16, 16, 64)   256         conv2d_109[0][0]                 
__________________________________________________________________________________________